

```
# This is formatted as code
```

# DMA Fall 21

**Note** : This entire lab will be manually evaluated.

Name : 'Jun Park'
Collaborator : ''

# Lab 4: Neural Networks #


In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
!wget http://askoski.berkeley.edu/~zp/lab_4_training.csv
!wget http://askoski.berkeley.edu/~zp/lab_4_test.csv

df_train = pd.read_csv('./lab_4_training.csv')
df_test = pd.read_csv('./lab_4_test.csv')
df_train.head()

--2021-09-22 03:35:01--  http://askoski.berkeley.edu/~zp/lab_4_training.csv
Resolving askoski.berkeley.edu (askoski.berkeley.edu)... 169.229.192.179
Connecting to askoski.berkeley.edu (askoski.berkeley.edu)|169.229.192.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79177 (77K) [text/csv]
Saving to: ‘lab_4_training.csv’

lab_4_training.csv  100%[===================>]  77.32K  --.-KB/s    in 0.1s    

2021-09-22 03:35:01 (788 KB/s) - ‘lab_4_training.csv’ saved [79177/79177]

--2021-09-22 03:35:02--  http://askoski.berkeley.edu/~zp/lab_4_test.csv
Resolving askoski.berkeley.edu (askoski.berkeley.edu)... 169.229.192.179
Connecting to askoski.berkeley.edu (askoski.berkeley.edu)|169.229.192.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26519 (26K) [text/csv]
Saving to: ‘lab_4_test.csv’

lab_4_test.csv      100%[===================>]  25.90K   107KB/s    in 0.2s    

2021-09-22 03:35:06 (107 KB/s) - ‘lab_4_test.csv’ saved [26519/2651

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv
0,577,male,20,third,hazel,72.0,180.0,0,0,5.0,No,0.0,100.0,10.0,10.0
1,677,male,19,second,hazel,72.0,120.0,1,1,16.0,Yes,9.0,70.0,3.0,5.0
2,1738,male,20,second,brown,63.0,55.0,1,2,15.0,Yes,4.5,15.0,4.0,13.0
3,1355,male,20,third,green,78.0,200.0,0,0,10.0,Yes,9.0,20.0,10.0,10.0
4,891,female,19,second,green,67.0,280.0,2,0,4.0,Yes,2.0,164.0,0.0,2.0


***
### Question 1###
Calculate a baseline accuracy measure using the majority class, assuming a target variable of 'gender'. The majority class is the most common value of the target variable in a particular dataset. Accuracy is calculated as (true positives + true negatives) / (all negatives and positives)

**Question 1.a**  
Find the majority class in the training set. If you always predicted this class in the training set, what would your accuracy be?

In [3]:
majority_class = df_train.groupby('gender').size()
majority_class

gender
female    647
male      545
dtype: int64

In [4]:
(647) / (647 + 545)

0.5427852348993288

###ANSWER: 0.5427852348993288
 

**Question 1.b**   
If you always predicted this same class (majority from the training set) in the test set, what would your accuracy be?

In [5]:
majority_test = df_test.groupby('gender').size()
majority_test

gender
female    208
male      190
dtype: int64

In [6]:
(208) / (208 + 190)

0.5226130653266332

###ANSWER: 0.5226130653266332


***
### Question 2 ###
Get started with Neural Networks.

   
Choose a NN implementation (eg: scikit-learn) and specify which you choose. Be sure the implementation allows you to modify the number of hidden layers and hidden nodes per layer.  

NOTE: When possible, specify the logsig (sigmoid/logistc) function as the transfer function (another word for activation function) and use Levenberg-Marquardt backpropagation (lbfgs). It is possible to specify logistic in Sklearn MLPclassifier (Neural net).  

**Question 2.a**   
Train a neural network with a single 10 node hidden layer. Only use the Height feature of the dataset to predict the Gender. You will have to change Gender to a 0 and 1 class. After training, use your trained model to predict the class using the height feature from the training set. What was the accuracy of this prediction?

In [7]:
x_train = df_train[['height']]
y_train = pd.get_dummies(df_train[['gender']])
mlp_class = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)
mlp_class.fit(x_train, y_train)
mlp_class.score(x_train, y_train)

0.8439597315436241

###ANSWER: 0.8439597315436241

**Question 2.b**  
Take the trained model from question 2.a and use it to predict the test set. This can be accomplished by taking the trained model and giving it the Height feature values from the test set. What is the accuracy of this model on the test set?

In [8]:
x_test = df_test[['height']]
y_test = pd.get_dummies(df_test[['gender']])

mlp_class.score(x_test, y_test)

0.8542713567839196

###ANSWER: 0.8542713567839196

**Question 2.c**   
Neural Networks tend to prefer smaller, normalized feature values. Try taking the log of the height feature in both training and testing sets or use a Standard Scalar operation in SKlearn to centre and normalize the data between 0-1 for continuous values. Repeat question 2.a and 2.b with the log version or the normalized and centered version of this feature

In [9]:
mlp_class_c = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)
scaler_train = StandardScaler()
scaler_test = StandardScaler()

scaler_train.fit(x_train)
scaler_test.fit(x_test)

x_train['height'] = scaler_train.transform(x_train)
x_test['height'] = scaler_test.transform(x_test)

mlp_class_c.fit(x_train, y_train)

display(
mlp_class_c.score(x_train, y_train),
mlp_class_c.score(x_test, y_test)
)

0.8439597315436241

0.8542713567839196

###ANSWER: Training Set: 0.8439597315436241 // Testing Set: 0.8542713567839196

***

### Question 3###
The rest of features in this dataset barring a few are categorical. No ML method accepts categorical features, so transform year, eyecolor, exercise into a set of binary features, one feature per unique original feature value, and mark the binary feature as ‘1’ if the feature value matches the original value and ‘0’ otherwise. Using only these binary variable transformed features, train and predict the class of the test set. What was your accuracy using Neural Network with a single 10 node hidden layer? During training, use a maximum number of iterations of 50.

In [10]:
mlp_class_3 = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 10, max_iter = 50, activation = 'logistic', random_state = 42)

train_3 = df_train[['year', 'eyecolor', 'exercise']]
test_3 = df_test[['year', 'eyecolor', 'exercise']]

train_3_getdum = pd.get_dummies(train_3)
test_3_getdum = pd.get_dummies(test_3)

mlp_class_3.fit(train_3_getdum, y_train)
mlp_class_3.score(test_3_getdum, y_test)

0.5703517587939698

###ANSWER: 

***
### Question 4###
Using a NN, report the accuracy on  the test set of a model that trained only on the height and the eye color features of instances in the training set.

**Question 4.a**  
What is the accuracy on the test set using the original height values (no pre-processing) and eye color as a one-hot?

In [11]:
mlp_class_4a = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)
train_4a = pd.get_dummies(df_train[['eyecolor']])
train_4a['height'] = df_train['height']

test_4a = pd.get_dummies(df_test[['eyecolor']])
test_4a['height'] = df_test['height']

mlp_class_4a.fit(train_4a, y_train)
mlp_class_4a.score(test_4a, y_test)

0.8517587939698492

###ANSWER: 0.8618090452261307


**Question 4.b**  
What is the accuracy on the test set using the log of height values (applied to both training and testing sets) and eye color as a one-hot?

In [12]:
mlp_class_4b = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)
train_4b = pd.get_dummies(df_train[['eyecolor']])
train_4b['height'] = np.log(df_train['height'])

test_4b = pd.get_dummies(df_test[['eyecolor']])
test_4b['height'] = np.log(df_test['height'])

mlp_class_4b.fit(train_4b, y_train)
mlp_class_4b.score(test_4b, y_test)

0.8467336683417085

###ANSWER: 0.8492462311557789

**Question 4.c**  
What is the accuracy on the test set using the Z-score of height values and eye color as a one-hot? 

Z-score is a normalization function. It is the value of a feature minus the average value for that feature (in the training set), divided by the standard deviation of that feature (in the training set). Remember that, whenever applying a function to a feature in the training set, it also has to be applied to that same feature in the test set.

In [13]:
mlp_class_4c = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)

def z(col):
  avg = np.mean(col)
  sd = np.std(col)
  return (col - avg) / sd

train_4b['height'] = z(df_train['height'])
test_4b['height'] = z(df_test['height'])

mlp_class_4c.fit(train_4b, y_train)
mlp_class_4c.score(test_4b, y_test)

0.8693467336683417

###ANSWER: 0.8693467336683417

***
### Question 5 ###
Repeat question 4 for exercise hours + eye color

In [14]:
mlp_class_5a = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)
train_5a = pd.get_dummies(df_train[['eyecolor']])
train_5a['exercisehours'] = df_train['exercisehours']

test_5a = pd.get_dummies(df_test[['eyecolor']])
test_5a['exercisehours'] = df_test['exercisehours']

mlp_class_5a.fit(train_5a, y_train)
print('Accuracy Score for Exercise Hours + Eye Color: ', mlp_class_5a.score(test_5a, y_test))


Accuracy Score for Exercise Hours + Eye Color:  0.535175879396985


In [15]:
mlp_class_5b = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)
train_5b = pd.get_dummies(df_train[['eyecolor']])
train_5b['exercisehours'] = np.log(df_train['exercisehours']).replace([np.inf, -np.inf], np.nan)
train_5b = train_5b.dropna()
y_train_dropped = y_train.iloc[train_5b.index,:]

test_5b = pd.get_dummies(df_test[['eyecolor']])
test_5b['exercisehours'] = np.log(df_test['exercisehours']).replace([np.inf, -np.inf], np.nan)
test_5b = test_5b.dropna()
y_test_dropped = y_test.iloc[test_5b.index,:]

mlp_class_5b.fit(train_5b, y_train_dropped)
print('Accuracy Score for Log(Exercise Hours) + Eye Color: ', mlp_class_5b.score(test_5b, y_test_dropped))

Accuracy Score for Log(Exercise Hours) + Eye Color:  0.5690376569037657


In [16]:
mlp_class_5c = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)

def z(col):
  avg = np.mean(col)
  sd = np.std(col)
  return (col - avg) / sd

train_5a['exercisehours'] = z(df_train['exercisehours'])
test_5a['exercisehours'] = z(df_test['exercisehours'])

mlp_class_5c.fit(train_5a, y_train)
print('Accuracy Score for Z Score Exercise Hours + Eye Color: ', mlp_class_5c.score(test_5a, y_test))

Accuracy Score for Z Score Exercise Hours + Eye Color:  0.5653266331658291


###ANSWER: 
### Accuracy Score for Exercise Hours + Eye Color:  0.5603015075376885
### Accuracy Score for Log(Exercise Hours) + Eye Color:  0.5732217573221757 
### Accuracy Score for Z Score Exercise Hours + Eye Color:  0.5653266331658291

***
### Question 6###
Combine the features from question 3, 4, and 5 (year, eyecolor, exercise, height, exercise hours). For numeric features use the best normalization method from questions 4 and 5.

**Question 6.a**  
What was the NN accuracy on the test set using the single 10 node hidden layer?

In [17]:
mlp_class_6 = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)

train_6 = pd.get_dummies(df_train[['year', 'eyecolor', 'exercise']])
train_6['height'] = z(df_train['height'])
train_6['exercisehours'] = np.log(df_train['exercisehours']).replace([np.inf, -np.inf], np.nan)
train_6 = train_6.dropna()
y_train_6 = y_train.iloc[train_6.index,:]

test_6 = pd.get_dummies(df_test[['year', 'eyecolor', 'exercise']])
test_6['height'] = z(df_test['height'])
test_6['exercisehours'] = np.log(df_test['exercisehours']).replace([np.inf, -np.inf], np.nan)
test_6 = test_6.dropna()
y_test_6 = y_test.iloc[test_6.index,:]

mlp_class_6.fit(train_6, y_train_6)
mlp_class_6.score(test_6, y_test_6)

0.7949790794979079

###ANSWER: NN Accuracy Score: 0.799163179916318


***
### Question 7- Bonus (10%)###
Can you improve your test set prediction accuracy by 5% or more?  

See how close to that milestone of improvement you can get by modifying the tuning parameters of  Neural Networks (the number of hidden layers, number of hidden nodes in each layer, the learning rate aka mu). A great guide to tuning parameters is explained in this guide: http://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf. 

While the guide is specific to SVM and in particular the C and gamma parameters of the RBF kernel, the method applies to generally to any ML technique with tuning parameters.

Please also write a paragraph in a markdown cell below with an explanation of your approach and evaluation metrics.


In [18]:
mlp_class_7 = MLPClassifier(solver = 'lbfgs', max_iter = 21, hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)

mlp_class_7.fit(train_6, y_train_6)
mlp_class_7.score(test_6, y_test_6)

0.8493723849372385

In [19]:
acc = []
best_i = 1
best_score = 0

for i in np.arange(1, 100):
  mlp_class_7 = MLPClassifier(solver = 'lbfgs', max_iter = i, hidden_layer_sizes = 10, activation = 'logistic', random_state = 42)
  mlp_class_7.fit(train_6, y_train_6)
  score_add = mlp_class_7.score(test_6, y_test_6)
  acc.append(score_add)
  if (score_add > best_score):
    best_score = score_add
    best_i = i

best_i

21

In [20]:
acc[best_i]

0.8493723849372385

###ANSWER: 0.8493723849372385
